In [1]:
import os
os.chdir("../../")

import pandas as pd
import numpy as np
from datetime import datetime

import re
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

# Gensim/Spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel 
import spacy

from src.text.utils import *
from src.text.lda import *

In [2]:
target = os.getcwd() + "/data/text/solomon_islands/"
news_path = [filename for filename in os.listdir(target)  if "news" in filename and "ner" not in filename]

In [3]:
news = pd.DataFrame()
for path in news_path:
    temp = pd.read_csv(target + path).drop("Unnamed: 0", axis=1)
    temp["from"] = path.replace("_news.csv", "")
    if news.empty:
        news = temp
    else: 
        news = pd.concat([news, temp], axis=0)

news = news[news.news.isna() != True].reset_index(drop=True)

In [4]:
def convert_datetime_format(date_str):
    try:
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        # Convert it to the desired format
        return date_obj.strftime('%Y-%m-%d')
    except ValueError:
        return None

## LDA (mallet)

In [5]:
en = spacy.load('en_core_web_sm')
spacy_stopwords = list(en.Defaults.stop_words)

In [6]:
nltk_stopwords = stopwords.words("english")
nltk_unique = [sw for sw in nltk_stopwords if sw not in spacy_stopwords]
spacy_stopwords.extend(nltk_unique)

In [7]:
data = news.news.values.tolist()
data_words = list(sent_to_words(data))

In [8]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[data_words], min_count=5, threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

texts_preprocessed = preprocess_text(data_words, spacy_stopwords, bigram_mod,
                                     trigram_mod, nlp)

Stopwords has been done.


In [9]:
id2word = corpora.Dictionary(texts_preprocessed)
texts = texts_preprocessed
corpus = [id2word.doc2bow(text) for text in texts]

In [10]:
from gensim.models.wrappers import LdaMallet
mallet_path = os.getcwd() + '/../mallet-2.0.8/bin/mallet'
model_list, coherence_values = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=texts_preprocessed,
                                                        mallet_path=mallet_path,
                                                        start=2,
                                                        limit=20,
                                                        step=1)

Mallet LDA: 2 topics, 1 topic bits, 1 topic mask
Data loaded.
max tokens: 2592
total tokens: 4283306
<10> LL/token: -8.39367
<20> LL/token: -8.31068
<30> LL/token: -8.14915
<40> LL/token: -8.05094

0	25	police community project year support development service officer report provide case public area work student school include health country court 
1	25	government people country member provincial year work issue time national province good leader team add week day island visit today 

<50> LL/token: -7.99978
<60> LL/token: -7.96925
<70> LL/token: -7.95032
<80> LL/token: -7.93778
<90> LL/token: -7.92827

0	25	police community project year support officer service provide report area work public case student school include covid health development court 
1	25	government people country member issue provincial year national work good time add leader team province woman island day week today 

<100> LL/token: -7.92275
<110> LL/token: -7.91681
<120> LL/token: -7.91252
<130> LL/token: -7.9075


<980> LL/token: -7.8267
<990> LL/token: -7.82766

0	25	police community people year public officer report time case student week yesterday school area covid health issue court follow day 
1	25	government country support work people development project provincial national island year woman include provide member leader important program service plan 

<1000> LL/token: -7.82863

Total time: 58 seconds
Mallet LDA: 3 topics, 2 topic bits, 11 topic mask
Data loaded.
max tokens: 2592
total tokens: 4283306
<10> LL/token: -8.77049
<20> LL/token: -8.4607
<30> LL/token: -8.171
<40> LL/token: -8.07206

0	16.66667	police public officer report case issue court year company medium member man yesterday people call operation time law charge order 
1	16.66667	government country support development project work island provide include national community people policy leader ensure fund plan important service business 
2	16.66667	people year student work team school community week covid woman health day g

<660> LL/token: -7.93631
<670> LL/token: -7.93611
<680> LL/token: -7.9367
<690> LL/token: -7.93648

0	16.66667	police public officer report member case issue year court provincial medium company yesterday parliament election man call people operation statement 
1	16.66667	government country support development project work provide national include service island ensure plan fund policy leader important business address provincial 
2	16.66667	people community year student work team school covid health time week day good child country family woman water start today 

<700> LL/token: -7.93593
<710> LL/token: -7.9358
<720> LL/token: -7.93614
<730> LL/token: -7.93665
<740> LL/token: -7.93714

0	16.66667	police public officer report member case issue year court medium provincial company yesterday parliament election man call operation statement people 
1	16.66667	government country support development project work provide national include service ensure island fund plan policy leader importa

<310> LL/token: -8.00795
<320> LL/token: -8.00834
<330> LL/token: -8.00772
<340> LL/token: -8.00806

0	12.5	people service student community school project covid health work country area water year provide province week increase local include market 
1	12.5	country support development work woman island program training team include opportunity project year economic important visit community event provide develop 
2	12.5	police officer year case public court time report yesterday man people family day charge rsipf accuse leave high today find 
3	12.5	government issue provincial member people national company parliament election land leader add statement process state public province decision office official 

<350> LL/token: -8.0071
<360> LL/token: -8.00781
<370> LL/token: -8.00761
<380> LL/token: -8.00658
<390> LL/token: -8.00711

0	12.5	people service student community school project covid health work country area water provide year province week increase local include market 
1	12.5	

<870> LL/token: -8.00706
<880> LL/token: -8.0065
<890> LL/token: -8.00592

0	12.5	people service student community school covid health work country area project water provide year province week increase local include market 
1	12.5	country support development work woman project island program training team include opportunity year community economic visit important provide event develop 
2	12.5	police officer year case public court time report yesterday man people day family charge rsipf accuse leave high follow today 
3	12.5	government issue provincial member people national company leader parliament election land statement process state add province public decision office meeting 

<900> LL/token: -8.00649
<910> LL/token: -8.00634
<920> LL/token: -8.00621
<930> LL/token: -8.00656
<940> LL/token: -8.00615

0	12.5	people service student community school covid health work country area water year provide project week increase province include market local 
1	12.5	country support developm

...

<350> LL/token: -8.22849
<360> LL/token: -8.22725
<370> LL/token: -8.22679
<380> LL/token: -8.22634
<390> LL/token: -8.22701

0	2.63158	travel road operation vehicle week honiara public ship service transport passenger time add flight work vessel arrive officer safety driver 
1	2.63158	people member parliament leader good country add continue church bill time nation speak work peace serve respect important bring change 
2	2.63158	country island security region regional visit support meeting meet include cooperation official ramsi mission leader international assistance delegation state discuss 
3	2.63158	service health medical provide work hospital staff people support care year supply receive include equipment clinic centre patient nurse doctor 
4	2.63158	police officer court charge rsipf accuse case man allege year arrest victim investigation incident prosecution force suspect public matter allegedly 
5	2.63158	covid country case health response border continue people test vaccine co

<510> LL/token: -8.2252
<520> LL/token: -8.22562
<530> LL/token: -8.22639
<540> LL/token: -8.22503

0	2.63158	travel road operation vehicle public week honiara ship boat transport passenger service flight vessel officer time add arrive safety drive 
1	2.63158	people member parliament leader good country add continue church bill time nation speak serve peace work change respect important bring 
2	2.63158	country island region security regional visit support meeting meet include cooperation official ramsi mission leader assistance delegation agreement international state 
3	2.63158	service health medical provide staff hospital work support people care equipment supply include clinic patient centre year receive nurse add 
4	2.63158	police officer court charge rsipf case accuse man allege arrest year investigation victim incident prosecution force matter suspect public allegedly 
5	2.63158	covid country case border response health test vaccine people continue risk measure number community 

<660> LL/token: -8.21934
<670> LL/token: -8.21935
<680> LL/token: -8.21889
<690> LL/token: -8.21913

0	2.63158	travel road operation vehicle public honiara week ship boat transport passenger service officer flight vessel arrive safety time drive add 
1	2.63158	people member leader parliament good country add continue church bill time work nation speak peace serve important respect leadership change 
2	2.63158	country island security region visit regional support meeting meet include cooperation official ramsi mission assistance leader state international delegation issue 
3	2.63158	service health medical provide support work staff hospital people care receive equipment supply clinic patient centre include nurse week year 
4	2.63158	police court officer charge case rsipf accuse man allege year arrest investigation victim prosecution incident force matter public suspect allegedly 
5	2.63158	covid country case border response health people continue test vaccine community risk measure numb

<810> LL/token: -8.21944
<820> LL/token: -8.21918
<830> LL/token: -8.21859
<840> LL/token: -8.21928

0	2.63158	travel road operation vehicle honiara week public ship boat transport passenger officer service flight time vessel arrive safety add drive 
1	2.63158	people member leader parliament good country add continue church bill time speak work nation serve peace respect important change leadership 
2	2.63158	country island security region visit regional support meeting meet include official cooperation ramsi mission assistance leader issue delegation discuss state 
3	2.63158	service health medical provide support work hospital staff people care supply equipment include clinic receive patient centre nurse year doctor 
4	2.63158	police court officer charge case rsipf accuse man allege year arrest investigation prosecution victim matter incident public suspect force allegedly 
5	2.63158	covid country case border health people response test vaccine continue risk community measure public p

<960> LL/token: -8.219
<970> LL/token: -8.21865
<980> LL/token: -8.21794
<990> LL/token: -8.21761

0	2.63158	travel road operation vehicle week honiara public ship boat transport passenger officer service flight vessel arrive time safety add drive 
1	2.63158	people member leader parliament good country add continue church bill time nation work speak peace serve change respect important leadership 
2	2.63158	country island visit security region regional support meeting meet official include cooperation assistance ramsi mission leader international state delegation discuss 
3	2.63158	service health medical provide support work staff hospital people care equipment include supply receive clinic patient centre nurse year doctor 
4	2.63158	police court officer charge case rsipf accuse man allege year arrest investigation prosecution public matter victim incident suspect force allegedly 
5	2.63158	covid country case health border people response continue test vaccine risk community public mea

In [11]:
best_model = model_list[np.argmax(coherence_values)]
best_model.print_topics()

[(0,
  '0.043*"issue" + 0.021*"medium" + 0.018*"concern" + 0.017*"statement" + 0.016*"call" + 0.015*"report" + 0.015*"claim" + 0.013*"decision" + 0.013*"question" + 0.013*"raise"'),
 (1,
  '0.040*"court" + 0.030*"case" + 0.026*"charge" + 0.025*"year" + 0.024*"accuse" + 0.018*"allege" + 0.018*"man" + 0.015*"victim" + 0.015*"high" + 0.015*"matter"'),
 (2,
  '0.160*"government" + 0.073*"provincial" + 0.060*"province" + 0.037*"national" + 0.025*"member" + 0.021*"western" + 0.020*"executive" + 0.017*"people" + 0.016*"guadalcanal" + 0.016*"yesterday"'),
 (3,
  '0.067*"service" + 0.038*"health" + 0.025*"provide" + 0.023*"medical" + 0.015*"child" + 0.014*"hospital" + 0.012*"include" + 0.012*"care" + 0.010*"equipment" + 0.009*"clinic"'),
 (4,
  '0.030*"development" + 0.021*"economic" + 0.020*"country" + 0.017*"sector" + 0.016*"policy" + 0.016*"support" + 0.015*"develop" + 0.011*"key" + 0.011*"resource" + 0.010*"ensure"'),
 (5,
  '0.116*"police" + 0.067*"officer" + 0.027*"rsipf" + 0.022*"force" 

In [12]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

mallet_lda_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(best_model)
vis_data = gensimvis.prepare(mallet_lda_model, corpus, id2word)
ldaviz_path = os.getcwd() + "/outputs/text/solomon_islands/sib_ldavis.html"
pyLDAvis.save_html(vis_data, ldaviz_path)
pyLDAvis.display(vis_data)

/Users/czhang/miniforge3/envs/po/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/czhang/miniforge3/envs/po/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/czhang/miniforge3/envs/po/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


## HDP/CTM

In [15]:
from gensim.models import CoherenceModel, HdpModel
hdpmodel = HdpModel(corpus=corpus, id2word=id2word)

In [158]:
from sklearn.feature_extraction.text import CountVectorizer
import scipy
from corextopic import corextopic as ct
vectorizer = CountVectorizer(
    stop_words=spacy_stopwords,
    ngram_range=(1,3),
    binary=True
)

In [167]:
word_doc = vectorizer.fit_transform(news["news"])
word_doc = scipy.sparse.csr_matrix(word_doc)

In [143]:
keywords = [["economics", "development", "economy"], ["tourism", "visit"],
            ["community"], ["government", "politics", "election"],
            ["food", "price", "security", "inflation"], ["police"],
            ["labor", "job", "employment", "work"]]

In [165]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = vectorizer.vocabulary_.keys()

In [168]:
corrtopics_model = ct.Corex(n_hidden=len(keywords))
corrtopics_model.fit(word_doc, anchors=keywords, words=words)